# Prerequisites

You need to have the `powerlaw` package installed in order to run any of the following. Installation is easy through `pip`:

In [ ]:
!pip install powerlaw

# Analysis

## Basic Setup

In [ ]:
import data # local module
import resulthelpers # local module
import powerlaw
import numpy
numpy.seterr(divide="ignore", invalid="ignore") # https://github.com/jeffalstott/powerlaw/issues/25

## Parameter Selection

For a given dataset (namely the CCA block and CCA raster datasets), we want to find the parameter values for which the power law fit is least sensitive. So we run fits on each possible set of parameters and then check for the mean and standard deviation of the $\alpha$ and $x_\text{min}$ values while holding a given parameter value constant.

In [ ]:
def compare_params(get_params, get_data, discrete):
    from pandas import DataFrame
    records = []
    params = get_params()
    for args in params.itertuples(index=False):
        data = get_data(*args)
        fit = powerlaw.Fit(data["population"], discrete=discrete, estimate_discrete=True, verbose=False)
        record = args._asdict()
        record["alpha"] = fit.power_law.alpha
        record["xmin"] = fit.xmin
        record["n"] = (data["population"] >= fit.xmin).sum()
        records.append(record)
    
    return DataFrame.from_records(records)

%time param_results = compare_params(data.get_cca_raster_params, data.get_cca_raster_data, discrete=False)

In [ ]:
param_results.groupby("dmin").agg({ "alpha": ["mean", "std"], "xmin": ["mean", "std"] })

In [ ]:
param_results.groupby("l").agg({ "alpha": ["mean", "std"], "xmin": ["mean", "std"] })

## Defining the Datasets

In [ ]:
# Datasets to process
# Beware: this variable is used in a few cells below.
datasets = {
    "CCA Tract, 1990/2000": {
        "get_data": data.get_cca_tract_19902000_data,
        "params": [3000],
        "columns": ["population", "area"],
        "discrete": ["population"]
    },
    "CCA Tract (2+), 1990/2000": {
        "get_data": data.get_cca_tract_19902000_data,
        "params": [3000, 2],
        "columns": ["population", "area"],
        "discrete": ["population"]
    },
    "CCA Tract (3+), 1990/2000": {
        "get_data": data.get_cca_tract_19902000_data,
        "params": [3000, 3],
        "columns": ["population", "area"],
        "discrete": ["population"]
    },
    "CCA Tract": {
        "get_data": data.get_cca_tract_data,
        "params": [3000],
        "columns": ["population", "area", "jobs"],
        "discrete": ["population", "jobs"]
    },
    "CCA Tract (2+)": {
        "get_data": data.get_cca_tract_data,
        "params": [3000, 2],
        "columns": ["population", "area", "jobs"],
        "discrete": ["population", "jobs"]
    },
    "CCA Tract (3+)": {
        "get_data": data.get_cca_tract_data,
        "params": [3000, 3],
        "columns": ["population", "area", "jobs"],
        "discrete": ["population", "jobs"]
    },
   "CCA Raster": {
        "get_data": data.get_cca_raster_data,
        "params": [1000, 5],
        "columns": ["population", "area", "jobs"],
        "discrete": []
    },
    "CCA Block": {
        "get_data": data.get_cca_block_data,
        "params": [1000, 1000],
        "columns": ["population", "area", "jobs"],
        "discrete": ["population", "jobs"]
    },
    "CCA Street Network": {
        "get_data": data.get_cca_street_network_data,
        "params": ["full", 100000],
        "columns": ["population", "area", "jobs"],
        "discrete": []
    },
    "Census UA/UC": {
        "get_data": data.get_census_uauc_data,
        "params": [],
        "columns": ["population", "area", "jobs"],
        "discrete": ["population", "jobs"]
    },
    "Census Place": {
        "get_data": data.get_census_place_data,
        "params": [],
        "columns": ["population", "area", "jobs"],
        "discrete": ["population", "jobs"]
    },
    "Census CBSA": {
        "get_data": data.get_census_cbsa_data,
        "params": [],
        "columns": ["population", "area", "jobs"],
        "discrete": ["population", "jobs"]
    },
    "Census CBSA, 2000": {
        "get_data": data.get_census_cbsa_2000_data,
        "params": [],
        "columns": ["population"],
        "discrete": ["population"]
    },
    "Census CBSA, 1990": {
        "get_data": data.get_census_cbsa_1990_data,
        "params": [],
        "columns": ["population"],
        "discrete": ["population"]
    }
}

## Overview of the Datasets

In [ ]:
def print_dataset_summary(dataset):
    df = dataset["get_data"](*dataset["params"])
    print("    number of cities: " + str(len(df)))
    for col in dataset["columns"]:
        print("  " + col)
        print("    sum: " + str(df[col].sum()))
        print("    min: " + str(df[col].min()))
        print("    max: " + str(df[col].max()))
    print("")

In [ ]:
for name, dataset in datasets.items():
    print(name)
    print_dataset_summary(dataset)

## Fitting the Models

For each dataset (and each column within that dataset), we run separate fits for:

- Upper tail: power law and lognormal
    - $x_\text{min}$ is determined by minimal KS for power law fit.
- Untruncated distribution: power law and lognormal
    - $x_\text{min}$ is explicitly set to the minimum value in the dataset.
    - If the dataset takes only discrete (integer) values, $x_\text{min}$ is set to $max \{ 10, x_\text{min} \}$ to avoid discrete estimation error. We call these results "semitruncated."

Since this produces a lot of results—and since we want graphs for everything too—we output all the results to files in `./results`. Once this is done, you can pull it back into a Pandas dataframe below for easier viewing.

Before running this, you may want to ensure that a `results` directory actually exists. This takes almost a half hour for me to run from within a small VirtualBox VM. Most of that time was spent on the Natural Cities data. You can skip the slow datasets by commenting them out of the `datasets` definition.

In [ ]:
def fit_models(dataframe, datasetname, discrete, col="population", outputdir=".", untruncated=False, xmin=None):
    import os
    import re
    import matplotlib.pyplot as plt
    
    data = dataframe[col]
    
    semitruncated = False
    if xmin is None and untruncated:
        # per Clauset et al., discrete estimation is not super reliable with small numbers
        xmin = min(data[data > 0])
        if discrete and xmin < 10:
            xmin = 10
            semitruncated = True
    # else: calculate optimal xmin by PL fit
    
    results = powerlaw.Fit(data, discrete=discrete, estimate_discrete=True, xmin=xmin, verbose=False)
    
    if untruncated:
        fitpart = "full"
    elif not xmin is None:
        fitpart = "min" + str(xmin)
    else:
        fitpart = "tail"
    if semitruncated:
        fitpart += "-semi"
    discretestr = "discrete" if discrete else "continuous"
    filename = os.path.join(outputdir, re.sub("[^a-z0-9]", "", datasetname.lower()) + "-" + col + "-" + fitpart)
    with open(filename + ".tsv", "w") as f:
        # headers. pretty!
        f.write("batchid\tdataset\tcolumn\tfitpart\tcontinuity\tn_original\tn\tn_tail\tx_min\talpha\tplsigma\tmu\tlnsigma\tlratio\tp_lratio\tks_pl\tks_ln\n")
        
        n_tail = (data >= results.xmin).sum()
        batchid = resulthelpers.batch_id(name=datasetname, col=col, alpha=results.power_law.alpha,
                                         x_min=results.xmin, n=len(data), n_tail=n_tail.item())
        
        f.write(batchid + "\t")
        f.write(datasetname + "\t")
        f.write(col + "\t")
        f.write(fitpart + "\t")
        f.write(discretestr + "\t")
        f.write(str(data.count()) + "\t")
        n = (data >= xmin).sum() if semitruncated else data.count() # n (may be less than n_original when semitruncated)
        f.write(str(n) + "\t")
        f.write(str(n_tail) + "\t")
        f.write(str(results.xmin) + "\t")
        f.write(str(results.power_law.alpha) + "\t")
        f.write(str(results.power_law.sigma) + "\t")
        f.write(str(results.lognormal.mu) + "\t")
        f.write(str(results.lognormal.sigma) + "\t")
        
        ratio, p = results.distribution_compare("power_law", "lognormal")
        f.write(str(ratio) + "\t") # pl:ln likelihood ratio
        f.write(str(p) + "\t") # likelihood ratio p-value
        
        f.write(str(results.power_law.KS()) + "\t")
        f.write(str(results.lognormal.KS()) + "\n")
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_xlabel(datasetname + " " + col)
    ax.set_ylabel("CCDF")
    ccdf = results.ccdf()
    ax.plot(ccdf[0], ccdf[1], '.', color="#00000030", markersize=8, markeredgewidth=0)
    
    # PL and LN:
    #results.power_law.plot_ccdf(ax=ax, color="#ff6600", linewidth=1.5, label="PL")
    #results.lognormal_positive.plot_ccdf(ax=ax, color="#0099cc", linewidth=1.5, label="LN")
    #plt.legend()
    
    # PL only:
    results.power_law.plot_ccdf(ax=ax, color="#ff6600", linewidth=1.5)
    
    fig.savefig(filename + ".png", bbox_inches="tight", dpi=300)
    fig.savefig(filename + ".pdf", bbox_inches="tight")
    plt.close()
    
    if fitpart == "tail":
        fig = plt.figure()
        ax = fig.add_subplot(111)
        ax.set_xlabel("$x_{min}$ (" + datasetname + " " + col + ")")
        ax.set_ylabel("α (±σ)")
        ax.set_ylim([0, 4])
        ax.set_xscale("log")
        ax.plot(results.xmins, results.alphas, color="#774499")
        ax.fill_between(results.xmins, results.alphas+results.sigmas, results.alphas-results.sigmas, color="#77449930", linewidth=0)
        ax.axhline(2, color="#000000a0", linewidth=1, linestyle="dotted")
        ax.axhline(results.power_law.alpha, color="#000000", linewidth=1)
        ax.axvline(results.xmin, color="#000000", linewidth=1)
        
        ax2 = ax.twinx()
        ax2.set_ylim([0, 1])
        ax2.set_ylabel("KS", color="#ee5555")
        ax2.tick_params("y", colors="#ee5555")
        ax2.plot(results.xmins, results.Ds, color="#ee5555")
        
        fig.savefig(filename + "-fits.png", bbox_inches="tight", dpi=300)
        fig.savefig(filename + "-fits.pdf", bbox_inches="tight")
        plt.close()

for name, dataset in datasets.items():
    df = dataset["get_data"](*dataset["params"])
    for col in dataset["columns"]:
        discrete = col in dataset["discrete"]
        fit_models(df, col=col, discrete=discrete, datasetname=name, outputdir="results")
        fit_models(df, col=col, discrete=discrete, datasetname=name, untruncated=True, outputdir="results")


In [ ]:
result_df = resulthelpers.load_basic_results("results/*.tsv")
result_df.sort_values(["dataset", "column", "fitpart"])

## Bootstrapping

To get a p-values for the power law fits, we perform the bootstrapping test described in Clauset et al. 2007. But that's way too slow to run in here.

Instead, we (crudely) generate batch files to run through SLURM on a high-performance cluster. Then once those results are run, we aggregate them into a single TSV, and load it back in here for viewing and comparing with the empirical data.

In [ ]:
def write_bootstrap_batch_files():
    # This is pretty ugly.
    i = 0
    for name, dataset in datasets.items():
        df = dataset["get_data"](*dataset["params"])
        for col in dataset["columns"]:
            i += 1
            args = []
            
            tail_result = result_df.query("dataset=='" + name + "' and column=='" + col + "' and fitpart=='tail'")
            xmin = tail_result["x_min"][0]
            
            batchid = str(tail_result["batchid"][0])
            args.append(str(tail_result["alpha"][0]))
            args.append(str(tail_result["x_min"][0]))
            args.append(str(tail_result["n_original"][0]))
            args.append(str(tail_result["n_tail"][0]))
            args.append(batchid)
            
            args.append("--trials 625")
            
            if col in dataset["discrete"]:
                args.append("--discrete")
            
            with open("batch/" + str(i) + ".sh", "w") as f:
                f.write("#!/bin/bash\n")
                f.write("#SBATCH --partition=batch\n")
                f.write("#SBATCH --cpus-per-task=20\n")
                f.write("#SBATCH --ntasks=4\n")
                if name != "CCA Street Network": # those take forever...
                    f.write("#SBATCH --time=6:00:00\n")
                f.write("#SBATCH --output=results/%j.out\n\n")
                f.write("# " + name + " " + col + "\n")
                
                f.write("srun /path/to/python simulate_powerlaw.py " + " ".join(map(str, args)))
            
            lower_tail = df[col][df[col] < xmin]
            numpy.save("batch/" + batchid + ".npy", lower_tail)

write_bootstrap_batch_files()